# Bosonic.py

In [1]:
import numpy as np
from scipy.special import binom 
from scipy.sparse import dok_matrix, linalg
from scipy import linalg as linalg_d
from joblib import Memory
import random
import plotly.graph_objects as go

location = './cachedir'
memory = Memory(location, verbose=0)

In [2]:
class fixed_basis:

    # Convierte a un enterno n a su escritura en base b
    def int_to_tuple(self, n, b, digits = None):
        rep = np.base_repr(n, b)
        rep_int = [int(x,b) for x in rep] 
        if digits is not None:
            zeros = [0 for i in range(0,digits-len(rep))]
            return zeros + rep_int
        else:
            return rep_int
    
    # Revierte la transformacion anterior
    def tuple_to_int(self, t):
        b = self.d-1
        l = len(t)
        s = [t[k]*b**(l-k-1) for k in range(0,l)]
        return sum(s)

    # Convierte el vector en su representacion
    def vect_to_repr(self, vect):
        for i, k in enumerate(vect):
            if k == 1. or k == 1:
                break
        return self.base[i,:]
            
    def rep_to_vect(self, rep):
        rep = list(rep)
        for i, r in [(j, self.base[j,:]) for j in range(0,self.size)]:
            if list(r) == rep:
                return self.canonicals[:,i]
        else:
            None
    
    def rep_to_index(self, rep):
        rep = list(rep)
        for i, r in [(j, self.base[j,:]) for j in range(0,self.size)]:
            if list(r) == rep:
                return i
        else:
            None
        
    # Crea base de M particulas en D estados (repr y base canonica)
    def create_basis(self, m, d):
        length = int(binom(d+m-1,d-1))
        base = []
        index = 0
        for x in range(0,(d+1)**max(d,m)):
            x = self.int_to_tuple(x,m+1,d)
            if sum(x) == m and len(x) == d:
                base.append(x)
                index += 1
        base = np.array(base)
        # Asignamos a cada uno de ellos un canónico
        x = [1.0 for j in range(0,length)]
        canonicals = np.diag(x)
        return base, canonicals
    
    def __init__(self, m, d):
        self.m = m
        self.d = d
        self.size = int(binom(d+m-1,d-1))
        (self.base, self.canonicals) = self.create_basis(m, d)

# Matrices de aniquilación y creación endomórficas. Estan fuera de la clase para poder ser cacheadas        
@memory.cache    
def bdb(basis, i, j):
    mat = dok_matrix((basis.size, basis.size), dtype=np.float32)
    if i != j:
        for k, v in enumerate(basis.base):
            if v[j] != 0:
                dest = list(v.copy())
                dest[j] -= 1
                dest[i] += 1
                tar = basis.rep_to_index(dest)
                mat[tar, k] = np.sqrt(v[i]+1)*np.sqrt(v[j])
    else:
        for k, v in enumerate(basis.base):
            if v[j] != 0:
                mat[k, k] = v[i] 
    return mat

@memory.cache    
def bbd(basis, i, j):
    mat = dok_matrix((basis.size, basis.size), dtype=np.float32)
    if i != j:
        for k, v in enumerate(basis.base):
            if v[i] != 0:
                dest = list(v.copy())
                dest[i] -= 1
                dest[j] += 1
                tar = basis.rep_to_index(dest)
                mat[tar, k] = np.sqrt(v[j]+1)*np.sqrt(v[i])
    else:
        for k, v in enumerate(basis.base):
            mat[k, k] = v[i]+1
    return mat

# Matrices de aniquilación y creación.Toman la base de origen y destino (basis_o, basis_d) resp
@memory.cache   
def b(basis_o, basis_d, i):
    mat = dok_matrix((basis_d.size, basis_o.size), dtype=np.float32)
    for k, v in enumerate(basis_o.base):
        if v[i] != 0:
            dest = list(v.copy())
            dest[i] -= 1   
            tar = basis_d.rep_to_index(dest)
            mat[tar, k] = np.sqrt(v[i])
    return mat

@memory.cache   
def bd(basis_o, basis_d, i):
    mat = dok_matrix((basis_d.size, basis_o.size), dtype=np.float32)
    for k, v in enumerate(basis_o.base):
        dest = list(v.copy())
        dest[i] += 1   
        tar = basis_d.rep_to_index(dest)
        mat[tar, k] = np.sqrt(v[i]+1)
    return mat

# Devuelve la matriz gamma asociada a la descomposición (M,N-M) del vector
def gamma(basis, m, vect, m_basis = None, nm_basis = None):
    d = basis.d
    if not m_basis or not nm_basis:
        m_basis = fixed_basis(m, d)
        nm_basis = fixed_basis(basis.m-m,d)
    mat = dok_matrix((m_basis.size, nm_basis.size), dtype=np.float32)
    for i, v in enumerate(m_basis.base):
        for j, w in enumerate(nm_basis.base):
            targ = v+w
            index = basis.rep_to_index(targ)
            if index == None:
                continue
            coef = vect[index]
            if coef != 0:      
                aux = lambda x: np.prod(np.reciprocal(np.sqrt([np.math.factorial(o) for o in x])))
                coef = coef * aux(v) * aux(w)
                #print(v,w,coef)
            mat[i,j] = coef
    return mat

# Devuelve la matriz rho M asociada al vector
def rho_m(basis, m, vect, m_basis = None, nm_basis = None):
    g = gamma(basis, m, vect, m_basis, nm_basis)
    return np.dot(g,np.transpose(g))

In [5]:
m = 4
d = 4
a = fixed_basis(m, d)
print(a.base)
a.rep_to_index(np.array([0, 1, 1]))

[[0 0 0 4]
 [0 0 1 3]
 [0 0 2 2]
 [0 0 3 1]
 [0 0 4 0]
 [0 1 0 3]
 [0 1 1 2]
 [0 1 2 1]
 [0 1 3 0]
 [0 2 0 2]
 [0 2 1 1]
 [0 2 2 0]
 [0 3 0 1]
 [0 3 1 0]
 [0 4 0 0]
 [1 0 0 3]
 [1 0 1 2]
 [1 0 2 1]
 [1 0 3 0]
 [1 1 0 2]
 [1 1 1 1]
 [1 1 2 0]
 [1 2 0 1]
 [1 2 1 0]
 [1 3 0 0]
 [2 0 0 2]
 [2 0 1 1]
 [2 0 2 0]
 [2 1 0 1]
 [2 1 1 0]
 [2 2 0 0]
 [3 0 0 1]
 [3 0 1 0]
 [3 1 0 0]
 [4 0 0 0]]


In [ ]:
# Prueba de las matrices gamma (TODO normalizacion)
v = a.rep_to_vect([0, 1, 1, 2])+a.rep_to_vect([1, 1, 2, 0])
print(v)
gamma(a, 2, v)
v = np.array([random.random() for a in range(0,a.size)])
v = v / np.linalg.norm(v)
print(v)
gamma(a, 2, v)

[0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
[0.29033901 0.13673684 0.08792772 0.13363623 0.08228274 0.2349543
 0.15559778 0.15495642 0.00073171 0.15029627 0.00290446 0.283889
 0.09512864 0.20174247 0.08405987 0.02414921 0.01972325 0.11700251
 0.28425527 0.1501779  0.04730217 0.1910561  0.29681086 0.27637613
 0.12939154 0.07349414 0.16606558 0.11792164 0.27328547 0.04051742
 0.14844461 0.24804708 0.18761501 0.14194226 0.03336991]


<10x10 sparse matrix of type '<class 'numpy.float32'>'
	with 100 stored elements in Dictionary Of Keys format>

In [ ]:
# Prueba de funcionamiento de las matrices bdb y bbd
v = [1, 1, 0, 2]
vect = a.rep_to_vect(v)
res = bbd(a,0,1) * vect
if np.linalg.norm(res) != 0:
    res = res / np.linalg.norm(res)
    print(a.vect_to_repr(res))
else:
    res = 0
    print(0)


[0 2 0 2]


In [ ]:
# Prueba de funcionamiento de las matrices b y bd

l = fixed_basis(3, 3)
m = fixed_basis(2, 3)

vect = l.rep_to_vect([0,2,1])
res = b(l,m,2) * vect
if np.linalg.norm(res) != 0:
    res = res / np.linalg.norm(res)
    #print(m.vect_to_repr(res))
    
vect = m.rep_to_vect([1,0,1])
res = bd(m,l,2) * vect
if np.linalg.norm(res) != 0:
    res = res / np.linalg.norm(res)
    print(l.vect_to_repr(res))

[1 0 2]


## Pairing

Construimos el espacio para n particulas en d estados. El hamiltoniano de pairing posee energias equiespaciadas con degeneración doble (asociada a cada estado y su inverso temporal), así como un término de interacción. Por tal motivo, crearemos un espacio con d = 2d.

In [3]:
m = 6
d = 3
# Creo las bases para no tener que recrearlas luego
basis = fixed_basis(m, 2*d)
basis_m1 = fixed_basis(m-1, 2*d)
basis_m2 = fixed_basis(m-2, 2*d)
print(basis.base)
print(basis.size)



[[0 0 0 0 0 6]
 [0 0 0 0 1 5]
 [0 0 0 0 2 4]
 ...
 [5 0 1 0 0 0]
 [5 1 0 0 0 0]
 [6 0 0 0 0 0]]
462


In [4]:
# Parametros hamiltoniano
e = 1
eps = 0.0001
e0 = np.zeros(2*d)
eigenspace_tol = 0.0001
for k in range(0, d):
    r = random.random() * eps
    e0[2*k] = k*e+r
    e0[2*k+1] = k*e+r

def hamiltonian(g, basis, basis_m1, basis_m2):
    # Construccion de H
    h0 = sum([e0[k]*bdb(basis,k,k) for k in range(0,2*d)])
    hi = dok_matrix((basis.size, basis.size), dtype=np.float32)
    for k in range(0,d):
        for kb in range(0,d):
            hi += -1*g*bd(basis_m1, basis, 2*k)*bd(basis_m2, basis_m1, 2*k+1)*b(basis_m1, basis_m2, 2*kb+1)*b(basis, basis_m1, 2*kb)

    return h0+hi

def solve(g, m, last_step = None):
    h = hamiltonian(g, basis, basis_m1, basis_m2)
    sol = linalg.eigsh(h,which='SA',k=19)
    if type(last_step) != type(None):
        # Seleccionamos todos los autovects que difieren sus autovalores menos que tol (mismo autoespacio)
        # y tomamos la proyección en el autoespacio de la solución del paso anterior (last_step)
        eig = sol[0].real
        eigv = sol[1]
        cand = [eigv[:,i].real  for (i, x) in enumerate(eig) if abs(x-min(eig)) < eigenspace_tol]
        cand_norm = [x/np.linalg.norm(x) for x in cand]
        fund = np.zeros(len(cand[0]))
        for x in cand_norm:
            fund += np.dot(last_step,x) * x
        #print(np.dot(fund,last_step))
    else:
        argmin = np.argmin(sol[0].real)
        fund = sol[1][:,argmin]
    fund = fund.real / np.linalg.norm(fund)
    return fund

In [5]:
# Rutina de resolución
m = 2
m_basis = fixed_basis(m, 2*d)
nm_basis = fixed_basis(basis.m-m, 2*d)

num = 200
g_range = np.linspace(0.01,5,num)

size = int(binom(2*d+m-1,2*d-1))
rho_range = np.zeros((num,size))
last_step = None
for i, g in enumerate(g_range):
    print(g)
    fund = solve(g, m, last_step)
    rho = rho_m(basis, m, fund, m_basis, nm_basis).todense()
    rho_range[i,:] = np.sort(linalg_d.eigvals(rho).real)
    last_step = fund

0.01
0.03507537688442211
0.060150753768844226
0.08522613065326633
0.11030150753768844
0.13537688442211057
0.16045226130653267
0.1855276381909548
0.2106030150753769
0.235678391959799
0.26075376884422113
0.28582914572864326
0.31090452261306534
0.33597989949748747
0.3610552763819096
0.38613065326633167
0.4112060301507538
0.4362814070351759
0.461356783919598
0.48643216080402013
0.5115075376884423
0.5365829145728643
0.5616582914572865
0.5867336683417086
0.6118090452261307
0.6368844221105529
0.6619597989949749
0.687035175879397
0.7121105527638192
0.7371859296482413
0.7622613065326633
0.7873366834170855
0.8124120603015076
0.8374874371859297
0.8625628140703518
0.8876381909547739
0.912713567839196
0.9377889447236182
0.9628643216080403
0.9879396984924623
1.0130150753768845
1.0380904522613066
1.0631658291457287
1.0882412060301507
1.113316582914573
1.138391959798995
1.1634673366834172
1.1885427135678392
1.2136180904522613
1.2386934673366834
1.2637688442211057
1.2888442211055278
1.3139195979899498


In [6]:
# Ploteamos
fig = go.Figure()
for x in range(0,size):
    fig.add_trace(go.Scatter(
        x=g_range,
        y=rho_range[:,x]
    ))
fig.update_layout(xaxis_title='G',
                   yaxis_title='Rho2')
fig.show()

In [ ]:
m = 3
d = 2
# Creo las bases para no tener que recrearlas luego
basis = fixed_basis(m, 2*d)
basis_m1 = fixed_basis(m-1, 2*d)
basis_m2 = fixed_basis(m-2, 2*d)
print(basis.base)
e = 1
eps = 0.0001
e0 = np.zeros(2*d)
for k in range(0, d):
    r = random.random() * eps
    e0[2*k] = k*e+r
    e0[2*k+1] = k*e+r

h = hamiltonian(2, basis, basis_m1, basis_m2)
ha = np.array(h.todense())
ha.astype(np.float32).tofile("myData.dat")
basis.size

[[0 0 0 3]
 [0 0 1 2]
 [0 0 2 1]
 [0 0 3 0]
 [0 1 0 2]
 [0 1 1 1]
 [0 1 2 0]
 [0 2 0 1]
 [0 2 1 0]
 [0 3 0 0]
 [1 0 0 2]
 [1 0 1 1]
 [1 0 2 0]
 [1 1 0 1]
 [1 1 1 0]
 [1 2 0 0]
 [2 0 0 1]
 [2 0 1 0]
 [2 1 0 0]
 [3 0 0 0]]


20

In [ ]:
import plotly.express as px

m = 2
num = 200
h = hamiltonian(2, basis, basis_m1, basis_m2)
sol = linalg.eigsh(h,which='SA',k=19,return_eigenvectors=True, v0=np.ones(20))
argmin = np.argmin(sol[0].real)
fund = sol[1][:,argmin]
fund = fund.real / np.linalg.norm(fund)
#rho = rho_m(basis, m, fund)
print(sol[0].real, fund)


[-5.4640498e+00 -5.4640489e+00 -3.8283179e+00 -3.8283174e+00
  3.5583711e-05  3.5830861e-05  1.0000726e+00  1.0000726e+00
  1.0000727e+00  1.0000731e+00  1.4641954e+00  1.4641956e+00
  1.8285362e+00  1.8285363e+00  2.0001082e+00  2.0001090e+00
  2.0001090e+00  2.0001090e+00  3.0001452e+00] [-5.3989545e-08  1.7978337e-07  1.7978337e-07 -5.3989545e-08
 -1.1485380e-08 -3.2505491e-01 -1.1485380e-08 -1.1485380e-08
 -1.1485380e-08 -5.3989545e-08 -1.1485380e-08 -3.2505491e-01
 -1.1485380e-08  1.1602712e-07  1.1602712e-07 -6.2796444e-01
 -1.1485380e-08 -1.1485380e-08 -6.2796444e-01 -5.3989545e-08]


In [ ]:
np.zeros(10)

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])

In [ ]:
h = hamiltonian(2, basis, basis_m1, basis_m2)
sol = linalg.eigsh(h,which='SA',k=19,return_eigenvectors=True)
eig = sol[0].real
eigv = sol[1]
print(eig)
cand = [eigv[:,i].real for (i, x) in enumerate(eig) if abs(x-min(eig)) < 0.0001]
cand_norm = [x/np.linalg.norm(x) for x in cand]
fund = np.zeros(len(cand[0]))
for x in cand_norm:
    fund += np.dot(last_step,x) * x
np.dot(cand[1],cand[0])

[-1.0593842e+01 -7.5822659e+00 -7.5822611e+00 -6.1228285e+00
 -6.1228261e+00 -6.1228256e+00 -6.1228251e+00 -5.5733442e+00
 -4.6053076e+00 -4.6053057e+00  3.1805006e-04  3.1894477e-04
  1.0002952e+00  1.0002953e+00  1.0002959e+00  1.0002961e+00
  1.5828569e+00  1.5828573e+00  2.0002732e+00]


ValueError: shapes (20,) and (35,) not aligned: 20 (dim 0) != 35 (dim 0)